In [1]:
import re
import PyPDF2
import os
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np

In [2]:
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

In [3]:
reader = PyPDF2.PdfReader("data/Instrukcje Aparatura Automatyzacji.pdf")

parts = []
for page in reader.pages[2:]:
    page.extract_text(visitor_text=visitor_body)
text = "".join(parts)

# replace special characters
text = re.sub(r'[^A-Za-z0-9 ,.\nęóąśłżźćńĘÓĄŚŁŻŹĆŃ]+', '', text)
# remove tables
text = re.sub(r'Tabela [0-9](.*?)\n \n', '', text, flags=re.DOTALL)
# remove white spaces and newlines
text = text.replace('\n', ' ')
text = re.sub(r'\s+', ' ', text)

df_fromdict = []
for num_task, content in enumerate(re.split(r'Ćwiczenie [0-9]', text)[1:]):
    content = re.sub(r'[0-9]', '', content)
    content = content.replace(' ', str(num_task))
    df_fromdict.append({
        'content': content[2:],
        'num_task': num_task,
        'type': 'instruction'
    })

# with open("preprocessed/instructions.txt", "w") as text_file:
#     text_file.write(''.join([d['content'] for d in df_fromdict]))

In [4]:
full_text = ''
for dir in next(os.walk('data'))[1]:
    for filename in os.listdir(f'data/{dir}'):
        num_task = re.findall('[0-9]', filename)[0]
        reader = PyPDF2.PdfReader(f"data/{dir}/{filename}")
        
        parts = []
        for page in reader.pages:
            page.extract_text(visitor_text=visitor_body)
        text = "".join(parts)
        
        # remove special characters
        text = re.sub(r'[^A-Za-z0-9 ,.\nęóąśłżźćńĘÓĄŚŁŻŹĆŃ]+', '', text)
        # remove blank lines
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text)
        # delete numbers, replace spaces with task num
        text = re.sub(r'[0-9]', '', text)
        text = text.replace(' ', num_task)
        # unify intro
        # text = 'Wstęp' + re.sub(r'.*(Wstęp|WSTĘP|Cel ćwiczenia|Opis ćwiczenia|Opis konfiguracji sprzętu)', '', text, flags=re.DOTALL)
        text = 'Wstęp' + re.sub(r'.*(Wst ep|Wstęp|WSTĘP|Cel ćwiczenia|Opis ćwiczenia|Opis konfiguracji sprzętu)', '', text, flags=re.DOTALL)
        full_text += text
        df_fromdict.append({
            'content': text,
            'num_task': int(num_task),
            'type': 'report'
        })

# with open("preprocessed/reports.txt", "w") as text_file:
#     text_file.write(full_text)

In [5]:
import pickle
pickle_out = open('kaggle_pickle/df_data.pkl', 'wb')
pickle.dump(df_fromdict, pickle_out)
pickle_out.close()